In [ ]:
# PATH
if (!$env:PATH.Contains(".crc/bin/oc")) { $env:PATH = $env:HOME+"/.crc/bin/oc:"+$env:PATH }
# as kubeadmin
oc login -u kubeadmin https://api.crc.testing:6443

$ns = oc config view --minify -o 'jsonpath={..namespace}'

In [ ]:
oc delete configmap main-sh
oc delete pod middleware-service

In [ ]:
oc create configmap main-sh --from-file=oc-api-client.sh

@"
apiVersion: v1
kind: Pod
metadata:
  name: middleware-service
  labels:
    environment: production
    app: nginx

# service account to read ns and pods
spec:
  serviceAccountName: egressips-reader

# folder served by web server
  volumes:
  - name: shared-data
    emptyDir: {}
  # sample file
  - name: dcobj-sample
    configMap:
      name: dcobj-sample
  # middleware script for worker
  - name: main-sh
    configMap:
        name: main-sh

  containers:

  - name: web
    image: nginx
    volumeMounts:
    
    - name: shared-data
      mountPath: /usr/share/nginx/html

  - name: worker
    image: ubuntu
    volumeMounts:
    - name: shared-data
      mountPath: /pod-data
    - mountPath: /usr/local/main
      name: main-sh
      readOnly: true
    command: ["/bin/bash"]
    args:
      - "-c"
      - "date | tee /pod-data/index.html; . /usr/local/main/oc-api-client.sh"
"@ | oc apply -f -

In [ ]:
oc describe pod middleware-service

In [ ]:
oc logs  middleware-service -c worker

# kubectl logs  middleware-service -c worker -f


In [ ]:
# expose web server on node port
oc expose pod middleware-service --type=NodePort --port=80

# minikube service --url middleware-service 

In [ ]:
# create new NS
oc create namespace lab

# start some services
1..5 | % {
    $index = $_
    oc run "web-$index" --image nginx -n lab
}

In [ ]:
oc get pods -A -o wide

In [ ]:
# remove lab pods
kubectl delete namespace lab

In [ ]:
# clean up
kubectl delete pod middleware-service

kubectl delete namespace lab

In [ ]:
$addr = minikube service --url middleware-service | cut -d "/" -f 3
$addr

In [ ]:
ngrok http $addr